In [1]:
# uncomment the install if they're not already installed
# installing rpy2 and loading so that I can use R within a python kernel if I start code cells with %%R
# !pip install rpy2
%load_ext rpy2.ipython
# !pip install kagglehub # so that I can the database

Error importing in API mode: ImportError("dlopen(/Users/minaheelkhan/anaconda3/lib/python3.11/site-packages/_rinterface_cffi_api.abi3.so, 0x0002): symbol not found in flat namespace '_R_BaseEnv'")
Trying to import in ABI mode.


# Not accurate, can't include dashed lines in dag model, we can replace all with our own diagram 

# tryna figure out NDE and NIE

## BMI as mediator in DSST case (Model 3)

In [121]:
%%R
# Binary outcome from DSST (example)
cutoff <- quantile(df$CFDDS, 0.25, na.rm = TRUE)
df$Y_lowDSST <- ifelse(df$CFDDS <= cutoff, 1, 0)


In [122]:
%%R
m2 <- subset(df, !is.na(Y_lowDSST) & !is.na(DRXTCAFF) & !is.na(RIDAGEYR) & !is.na(RIAGENDR))

fit_m2 <- glm(Y_lowDSST ~ DRXTCAFF + RIDAGEYR + RIAGENDR, family = binomial(), data = m2)
summary(fit_m2)

# Odds ratio for caffeine
OR_caff <- exp(coef(fit_m2)["DRXTCAFF"])
OR_caff


 DRXTCAFF 
0.9974669 


In [123]:
%%R
OR_caff_50 <- exp(50 * coef(fit_m2)["DRXTCAFF"])
OR_caff_50


 DRXTCAFF 
0.8808941 


In [124]:
%%R
# Keep complete cases for mediation
dat <- subset(df,
              !is.na(Y_lowDSST) & !is.na(DRXTCAFF) &
              !is.na(RIDAGEYR) & !is.na(RIAGENDR) &
              !is.na(BMXBMI))

# Mediator model: M ~ A + C
med_mod <- lm(BMXBMI ~ DRXTCAFF + RIDAGEYR + RIAGENDR, data = dat)

# Outcome model: Y ~ A + M + C
out_mod <- glm(Y_lowDSST ~ DRXTCAFF + BMXBMI + RIDAGEYR + RIAGENDR,
               family = binomial(), data = dat)


In [125]:
%%R
A0 <- mean(dat$DRXTCAFF)
A1 <- A0 + 50


In [126]:
%%R
set.seed(2026)
S <- 500  # increase to 2000 for final numbers

# Predicted mediator means under A0 and A1
muM_a0 <- predict(med_mod, newdata = transform(dat, DRXTCAFF = A0))
muM_a1 <- predict(med_mod, newdata = transform(dat, DRXTCAFF = A1))

# Residual SD for continuous mediator
sigma_m <- summary(med_mod)$sigma

# Draw M(a0) and M(a1)
M_a0 <- matrix(rnorm(nrow(dat) * S, mean = rep(muM_a0, S), sd = sigma_m), nrow = nrow(dat))
M_a1 <- matrix(rnorm(nrow(dat) * S, mean = rep(muM_a1, S), sd = sigma_m), nrow = nrow(dat))

# Function-free prediction loop: compute EY(a, Mdraw)
EY_am <- function(Aval, Mmat){
  vals <- numeric(S)
  for(s in 1:S){
    nd <- dat
    nd$DRXTCAFF <- Aval
    nd$BMXBMI   <- Mmat[, s]
    vals[s] <- mean(predict(out_mod, newdata = nd, type = "response"))
  }
  mean(vals)
}

EY_a0Ma0 <- EY_am(A0, M_a0)  # E[Y_{a0, M(a0)}]
EY_a1Ma0 <- EY_am(A1, M_a0)  # E[Y_{a1, M(a0)}]
EY_a0Ma1 <- EY_am(A0, M_a1)  # E[Y_{a0, M(a1)}]
EY_a1Ma1 <- EY_am(A1, M_a1)  # E[Y_{a1, M(a1)}]

# Risk difference scale (matches your Class 7 writeup)
TE  <- EY_a1Ma1 - EY_a0Ma0
NDE <- EY_a1Ma0 - EY_a0Ma0
NIE <- EY_a0Ma1 - EY_a0Ma0

c(EY_a0Ma0 = EY_a0Ma0, EY_a1Ma1 = EY_a1Ma1, TE = TE, NDE = NDE, NIE = NIE, check = NDE + NIE)


     EY_a0Ma0      EY_a1Ma1            TE           NDE           NIE 
 0.2430780177  0.2219566263 -0.0211213914 -0.0211108868 -0.0000111583 
        check 
-0.0211220451 


EYₐ₀Mₐ₀ = 0.24308
Expected risk of low cognition if caffeine is set to A₀ (baseline)
and BMI is allowed to take the value it would naturally have under A₀.
This is your baseline world.

EYₐ₁Mₐ₁ = 0.22196
Expected risk of low cognition if caffeine is set to A₁ (+50 mg/day)
and BMI is allowed to change naturally under A₁.
This is the fully intervened world.

TE
Increasing caffeine intake by 50 mg/day is associated with an approximately
2.1 percentage point reduction in the probability of low cognitive performance
when all pathways are allowed to operate naturally.

NDE
The change in cognitive risk when caffeine increases,
holding BMI at the level it would naturally have under baseline caffeine intake.
This isolates the non-BMI pathway

NIE
The portion of the caffeine effect that operates through BMI, very close to zero in this case

CHECK is just decomposition check to make sure TE = NDE + NIE

The mediation analysis suggests that the association between caffeine intake and cognitive performance is driven almost entirely by the direct pathway rather than by mediation through BMI. The estimated Natural Indirect Effect through BMI is effectively zero, while the Natural Direct Effect closely matches the Total Effect. This indicates that, within the assumptions of the mediation model, changes in BMI induced by caffeine intake do not meaningfully contribute to changes in cognitive performance. Conditioning on BMI therefore has little impact on the estimated association, consistent with the similarity between Models 3 and 4.

These estimates rely on strong assumptions, including no unmeasured confounding of the exposure–mediator, mediator–outcome, and exposure–outcome relationships, and correct model specification; they are therefore interpreted as sensitivity analyses rather than definitive causal effects.

In [127]:
%%R
x <- mean(dat$BMXBMI)

nd0 <- dat; nd1 <- dat
nd0$DRXTCAFF <- A0; nd1$DRXTCAFF <- A1
nd0$BMXBMI <- x;    nd1$BMXBMI <- x

EY_0x <- mean(predict(out_mod, newdata = nd0, type = "response"))
EY_1x <- mean(predict(out_mod, newdata = nd1, type = "response"))

CDE <- EY_1x - EY_0x
c(EY_0x = EY_0x, EY_1x = EY_1x, CDE = CDE)


      EY_0x       EY_1x         CDE 
 0.24323941  0.22213303 -0.02110638 


Holding BMI fixed, increasing caffeine intake by 50 mg/day is associated with an approximately 2.1 percentage point reduction in the probability of low cognitive performance.